In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization,Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# read data

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/train.csv')
train_label = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/train_labels.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/test.csv')

In [ ]:
train

In [ ]:
test

In [ ]:
train_label

# check data

In [ ]:
cols_with_missing = [col for col in train.columns if train[col].isnull().any()]
print('missing columns at train data is :',cols_with_missing)
cols_with_missing = [col for col in train_label.columns if train_label[col].isnull().any()]
print('missing columns at train_label data is :',cols_with_missing)
cols_with_missing = [col for col in test.columns if test[col].isnull().any()]
print('missing columns at test data is :',cols_with_missing)

**there is no missing data**

In [ ]:
print('----------train data type is----------')
print(train.dtypes)

print('----------train_label data type is----------')
print(train_label.dtypes)

print('----------test data type is----------')
print(test.dtypes)

**train and test data has only number imformation , no object data**

In [ ]:
train['label']=0
train.loc[train['sequence'].isin(train_label.loc[train_label['state']==1,'sequence']),'label']=1
train

In [ ]:
train['subject'].value_counts()/60

In [ ]:
test['subject'].value_counts()/60

**looks like the number of imformation at each subject have variation. some subject has over 100 samples but some others have less than 10**

In [ ]:
label_437=train.loc[train['subject']==437,'label'].value_counts()/60
label_10=train.loc[train['subject']==10,'label'].value_counts()/60
label_476=train.loc[train['subject']==486,'label'].value_counts()/60

print('----------subject 437 label ratio is----------')
print(label_437)
plt.pie(label_437,
        counterclock=False, startangle=90,
       autopct='%1.1f%%', pctdistance=0.7)
 
plt.show()

print('----------subject 10 label ratio is----------')
print(label_10)
plt.pie(label_10,
        counterclock=False, startangle=90,
       autopct='%1.1f%%', pctdistance=0.7)
 
plt.show()

print('----------subject 476 label ratio is----------')
print(label_476)
plt.pie(label_476,
        counterclock=False, startangle=90,
       autopct='%1.1f%%', pctdistance=0.7)
 
plt.show()

**ratio of state is different from each subject**

In [ ]:
print(train_label['state'].value_counts())
plt.pie(train_label['state'].value_counts(),
        counterclock=False, startangle=90,
       autopct='%1.1f%%', pctdistance=0.7)
 
plt.show()

**ratio of state is almost same on the whole**

In [ ]:
train_437=train.loc[train['subject']==437]
train_10=train.loc[train['subject']==10]
train_476=train.loc[train['subject']==476]
train_437[['sequence','label']].groupby('sequence').sum()/60

In [ ]:
label1=train_437[train_437['sequence']==4]
label0=train_437[train_437['sequence']==217]
label1_mean=train_437[train_437['label']==1].groupby('step').mean()
label0_mean=train_437[train_437['label']==0].groupby('step').mean()


print("time step at subjectt 437 data : ")
ncols = 3
nrows = 4
colum_list=label1.columns

fig, axes = plt.subplots(nrows, ncols, figsize=(40,20), facecolor='#EAEAF2')

for r in range(nrows):
    for c in range(ncols):
        col = colum_list[r*ncols+c+3]
        sns.lineplot(data=label1,ax=axes[r, c], x=label1.step,y=label1[col], color='red', label='label1')
        sns.lineplot(data=label0,ax=axes[r, c], x=label0.step,y=label0[col], color='blue', label='label0')
        sns.lineplot(data=label1_mean,ax=axes[r, c], x=label1_mean.index,y=label1_mean[col], color='green', label='label1_mean')
        sns.lineplot(data=label0_mean,ax=axes[r, c], x=label0_mean.index,y=label0_mean[col], color='pink', label='label0_mean')
plt.show()

In [ ]:
check=train_10[['sequence','label']].groupby('sequence').sum()/60
check.head()

In [ ]:
label1=train_10[train_10['sequence']==301]
label0=train_10[train_10['sequence']==606]
label1_mean=train_10[train_10['label']==1].groupby('step').mean()
label0_mean=train_10[train_10['label']==0].groupby('step').mean()


print("time step at subjectt 10 data : ")
ncols = 3
nrows = 4
colum_list=label1.columns

fig, axes = plt.subplots(nrows, ncols, figsize=(40,20), facecolor='#EAEAF2')

for r in range(nrows):
    for c in range(ncols):
        col = colum_list[r*ncols+c+3]
        sns.lineplot(data=label1,ax=axes[r, c], x=label1.step,y=label1[col], color='red', label='label1')
        sns.lineplot(data=label0,ax=axes[r, c], x=label0.step,y=label0[col], color='blue', label='label0')
        sns.lineplot(data=label1_mean,ax=axes[r, c], x=label1_mean.index,y=label1_mean[col], color='green', label='label1_mean')
        sns.lineplot(data=label0_mean,ax=axes[r, c], x=label0_mean.index,y=label0_mean[col], color='pink', label='label0_mean')
plt.show()

In [ ]:
train437_1_mean=train_437[train_437['label']==1].groupby('step').mean()
train10_1_mean=train_10[train_10['label']==1].groupby('step').mean()

print("time step at state 1 : ")
ncols = 3
nrows = 4
colum_list=label1.columns

fig, axes = plt.subplots(nrows, ncols, figsize=(40,20), facecolor='#EAEAF2')

for r in range(nrows):
    for c in range(ncols):
        col = colum_list[r*ncols+c+3]
        sns.lineplot(data=train437_1_mean,ax=axes[r, c], x=train437_1_mean.index,y=train437_1_mean[col], color='red', label='437')
        sns.lineplot(data=train10_1_mean,ax=axes[r, c], x=train10_1_mean.index,y=train10_1_mean[col], color='blue', label='10')
plt.show()

In [ ]:
train437_0_mean=train_437[train_437['label']==0].groupby('step').mean()
train10_0_mean=train_10[train_10['label']==0].groupby('step').mean()
train476_0_mean=train_476[train_476['label']==0].groupby('step').mean()

print("time step at state 0 : ")
ncols = 3
nrows = 4
colum_list=label1.columns

fig, axes = plt.subplots(nrows, ncols, figsize=(40,20), facecolor='#EAEAF2')

for r in range(nrows):
    for c in range(ncols):
        col = colum_list[r*ncols+c+3]
        sns.lineplot(data=train437_0_mean,ax=axes[r, c], x=train437_0_mean.index,y=train437_0_mean[col], color='red', label='437')
        sns.lineplot(data=train10_0_mean,ax=axes[r, c], x=train10_0_mean.index,y=train10_0_mean[col], color='blue', label='10')
        sns.lineplot(data=train476_0_mean,ax=axes[r, c], x=train476_0_mean.index,y=train476_0_mean[col], color='green', label='476')
plt.show()

In [ ]:
train

# search feature columns

In [ ]:
colum_list=train.drop(['sequence','subject','step','label'],axis=1).columns

In [ ]:
sequence_list=train.groupby(['sequence']).mean().index
train_data=train.copy()
train_data[['00','01','02','03','04','05','06','07','08','09','10','11','12']]=0

list_00_3=[]
list_01_3=[]
list_02_3=[]
list_03_3=[]
list_04_3=[]
list_05_3=[]
list_06_3=[]
list_07_3=[]
list_08_3=[]
list_09_3=[]
list_10_3=[]
list_11_3=[]
list_12_3=[]


for i in sequence_list:
    s=train[train['sequence']==i].rolling(3, min_periods=1).mean()
    list_00_3.extend(s[colum_list[0]].tolist())
    list_01_3.extend(s[colum_list[1]].tolist())
    list_02_3.extend(s[colum_list[2]].tolist())
    list_03_3.extend(s[colum_list[3]].tolist())
    list_04_3.extend(s[colum_list[4]].tolist())
    list_05_3.extend(s[colum_list[5]].tolist())
    list_06_3.extend(s[colum_list[6]].tolist())
    list_07_3.extend(s[colum_list[7]].tolist())
    list_08_3.extend(s[colum_list[8]].tolist())
    list_09_3.extend(s[colum_list[9]].tolist())
    list_10_3.extend(s[colum_list[10]].tolist())
    list_11_3.extend(s[colum_list[11]].tolist())
    list_12_3.extend(s[colum_list[12]].tolist())

train_data['00']=list_00_3
train_data['01']=list_01_3
train_data['02']=list_02_3
train_data['03']=list_03_3
train_data['04']=list_04_3
train_data['05']=list_05_3
train_data['06']=list_06_3
train_data['07']=list_07_3
train_data['08']=list_08_3
train_data['09']=list_09_3
train_data['10']=list_10_3
train_data['11']=list_11_3
train_data['12']=list_12_3

In [ ]:
train_10=train_data.loc[train_data['subject']==10]
label1_mean=train_10[train_10['label']==1].groupby('step').mean()
label0_mean=train_10[train_10['label']==0].groupby('step').mean()


print("time step at subjectt 10 data rolling 3 sec : ")
ncols = 3
nrows = 4
colum_list=label1_mean.columns

fig, axes = plt.subplots(nrows, ncols, figsize=(40,20), facecolor='#EAEAF2')

for r in range(nrows):
    for c in range(ncols):
        col = colum_list[r*ncols+c+16]
        sns.lineplot(data=label1_mean,ax=axes[r, c], x=label1_mean.index,y=label1_mean[col], color='red', label='label1_mean')
        sns.lineplot(data=label0_mean,ax=axes[r, c], x=label0_mean.index,y=label0_mean[col], color='blue', label='label0_mean')
plt.show()

In [ ]:
sequence_list=train.groupby(['sequence']).mean().index
train_data=train.copy()
train_data[['00','01','02','03','04','05','06','07','08','09','10','11','12']]=0

list_00_6=[]
list_01_6=[]
list_02_6=[]
list_03_6=[]
list_04_6=[]
list_05_6=[]
list_06_6=[]
list_07_6=[]
list_08_6=[]
list_09_6=[]
list_10_6=[]
list_11_6=[]
list_12_6=[]


for i in sequence_list:
    s=train[train['sequence']==i].rolling(6, min_periods=1).mean()
    list_00_6.extend(s[colum_list[0]].tolist())
    list_01_6.extend(s[colum_list[1]].tolist())
    list_02_6.extend(s[colum_list[2]].tolist())
    list_03_6.extend(s[colum_list[3]].tolist())
    list_04_6.extend(s[colum_list[4]].tolist())
    list_05_6.extend(s[colum_list[5]].tolist())
    list_06_6.extend(s[colum_list[6]].tolist())
    list_07_6.extend(s[colum_list[7]].tolist())
    list_08_6.extend(s[colum_list[8]].tolist())
    list_09_6.extend(s[colum_list[9]].tolist())
    list_10_6.extend(s[colum_list[10]].tolist())
    list_11_6.extend(s[colum_list[11]].tolist())
    list_12_6.extend(s[colum_list[12]].tolist())

train_data['00']=list_00_6
train_data['01']=list_01_6
train_data['02']=list_02_6
train_data['03']=list_03_6
train_data['04']=list_04_6
train_data['05']=list_05_6
train_data['06']=list_06_6
train_data['07']=list_07_6
train_data['08']=list_08_6
train_data['09']=list_09_6
train_data['10']=list_10_6
train_data['11']=list_11_6
train_data['12']=list_12_6

In [ ]:
train_10=train_data.loc[train_data['subject']==10]
label1_mean=train_10[train_10['label']==1].groupby('step').mean()
label0_mean=train_10[train_10['label']==0].groupby('step').mean()


print("time step at subjectt 10 data rolling 6 sec : ")
ncols = 3
nrows = 4
colum_list=label1_mean.columns

fig, axes = plt.subplots(nrows, ncols, figsize=(40,20), facecolor='#EAEAF2')

for r in range(nrows):
    for c in range(ncols):
        col = colum_list[r*ncols+c+16]
        sns.lineplot(data=label1_mean,ax=axes[r, c], x=label1_mean.index,y=label1_mean[col], color='red', label='label1_mean')
        sns.lineplot(data=label0_mean,ax=axes[r, c], x=label0_mean.index,y=label0_mean[col], color='blue', label='label0_mean')
plt.show()

**add some columns to train data**

In [ ]:
train['std']=train.std(axis=1)

train['01_03']=list_01_3
train['02_03']=list_02_3
train['03_03']=list_03_3

train['04_06']=list_04_6
train['05_06']=list_05_6
train

In [ ]:
sequence_list=test.groupby(['sequence']).mean().index

list_01=[]
list_02=[]
list_03=[]
list_04=[]
list_05=[]


for i in sequence_list:
    s=test[test['sequence']==i].rolling(3, min_periods=1).mean()
    list_01.extend(s[colum_list[1]].tolist())
    list_02.extend(s[colum_list[2]].tolist())
    list_03.extend(s[colum_list[3]].tolist())


for i in sequence_list:
    s=test[test['sequence']==i].rolling(6, min_periods=1).mean()
    list_04.extend(s[colum_list[4]].tolist())
    list_05.extend(s[colum_list[5]].tolist())
    
test['std']=test.std(axis=1)
test['01_03']=list_01
test['02_03']=list_02
test['03_03']=list_03
test['04_06']=list_04
test['05_06']=list_05

In [ ]:
test

In [ ]:
def target_scale(train_data):
    ss = preprocessing.StandardScaler()
    train_standardization = ss.fit_transform(train_data)
    return train_standardization

train_data=train.drop(['sequence','subject','label'],axis=1)
test_data=test.drop(['sequence','subject'],axis=1)
train_data=target_scale(train_data)
test_data=target_scale(test_data)

In [ ]:
def make_dataset(train_data,train_label):

    data, target = [], []
    time_step=60

    for i in range(len(train_label)):
        index=i*60
        data.append(train_data[index:index + time_step])
        target.append(train_label.loc[i,'state'])

    re_data = np.array(data).reshape(len(data), time_step, 20)
    re_target = np.array(target).reshape(len(data),1)

    return re_data, re_target

g, h=make_dataset(train_data,train_label)

In [ ]:
n_hidden = 150
model=None
model = keras.Sequential()
model = Sequential()
model.add(LSTM(n_hidden, batch_input_shape=(None,g.shape[1], g.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(180))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(75))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(30))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(10))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
optimizer = Adam(lr=0.001)
model.compile(loss='binary_crossentropy',metrics=['binary_accuracy'] ,optimizer=optimizer)

early_stopping = EarlyStopping(monitor='val_loss', mode='auto', min_delta=0.00001,patience=5,)
history=model.fit(g, h,
                  batch_size=500,
                  epochs=100,
                  validation_split=0.2,
                  callbacks=[early_stopping])

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

# make prediction for submit

In [ ]:
test_label=pd.DataFrame()
test_label['sequence']=test.groupby('sequence').mean().index
test_label['state']=0
test_label

In [ ]:
g_test,h_test= make_dataset(test_data,test_label)

In [ ]:
submission=np.round(model.predict(g_test))
test_label.state=submission
test_label['state']=test_label['state'].apply(int)
test_label

In [ ]:
print(test_label['state'].value_counts())
plt.pie(test_label['state'].value_counts(),
        counterclock=False, startangle=90,
       autopct='%1.1f%%', pctdistance=0.7)
 
plt.show()

In [ ]:
test_label.to_csv('submission.csv', index=False)